In [2]:
!pip install mysql

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 87 kB 5.1 MB/s             
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-082ypm9p/mysqlclient_edb9224768904429a78219d25889beef/setup.py'"'"'; __file__='"'"'/tmp/pip-install-082ypm9p/mysqlclient_edb9224768904429a78219d25889beef/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-udp57_f3
       cwd: /tmp/pip-install-082ypm9p/mysqlclient_edb9224768904429a78219d25889beef/
  Complete output (15 lines):
  /bin/sh: 1: mysql_config: not found
  /bin/sh: 1: mariadb_

     |████████████████████████████████| 87 kB 9.3 MB/s             
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-082ypm9p/mysqlclient_2024599f35dc48228c066916a662d532/setup.py'"'"'; __file__='"'"'/tmp/pip-install-082ypm9p/mysqlclient_2024599f35dc48228c066916a662d532/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-yg83tvhe
       cwd: /tmp/pip-install-082ypm9p/mysqlclient_2024599f35dc48228c066916a662d532/
  Complete output (12 lines):
  /bin/sh: 1: mysql_config: not found
  /bin/sh: 1: mariadb_config: not found
  /bin/sh: 1: mysql_config: not found
  Traceback (most rece

     |████████████████████████████████| 86 kB 7.7 MB/s             
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-082ypm9p/mysqlclient_0d700faa74df4325929f0bbc2e2f4314/setup.py'"'"'; __file__='"'"'/tmp/pip-install-082ypm9p/mysqlclient_0d700faa74df4325929f0bbc2e2f4314/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-gxm730by
       cwd: /tmp/pip-install-082ypm9p/mysqlclient_0d700faa74df4325929f0bbc2e2f4314/
  Complete output (12 lines):
  /bin/sh: 1: mysql_config: not found
  /bin/sh: 1: mariadb_config: not found
  /bin/sh: 1: mysql_config: not found
  Traceback (most rece

  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-082ypm9p/mysqlclient_4681bf79d4e24163a58c4ad89aa7894a/setup.py'"'"'; __file__='"'"'/tmp/pip-install-082ypm9p/mysqlclient_4681bf79d4e24163a58c4ad89aa7894a/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-vq387y9l
       cwd: /tmp/pip-install-082ypm9p/mysqlclient_4681bf79d4e24163a58c4ad89aa7894a/
  Complete output (10 lines):
  /bin/sh: 1: mysql_config: not found
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-install-082ypm9p/mysqlclient_4681bf79d4e24163a58c4ad89aa7894a/setup.py"

     |████████████████████████████████| 82 kB 379 kB/s             
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-082ypm9p/mysqlclient_031aed8e928b4a34a629ae4af51517dd/setup.py'"'"'; __file__='"'"'/tmp/pip-install-082ypm9p/mysqlclient_031aed8e928b4a34a629ae4af51517dd/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-6t977jfz
       cwd: /tmp/pip-install-082ypm9p/mysqlclient_031aed8e928b4a34a629ae4af51517dd/
  Complete output (10 lines):
  /bin/sh: 1: mysql_config: not found
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-ins

     |████████████████████████████████| 77 kB 10.7 MB/s            
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-082ypm9p/mysqlclient_5ea86a26c52b463a8feba09bb37d16a4/setup.py'"'"'; __file__='"'"'/tmp/pip-install-082ypm9p/mysqlclient_5ea86a26c52b463a8feba09bb37d16a4/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-vntxmwxr
       cwd: /tmp/pip-install-082ypm9p/mysqlclient_5ea86a26c52b463a8feba09bb37d16a4/
  Complete output (10 lines):
  /bin/sh: 1: mysql_config: not found
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-ins

     |████████████████████████████████| 76 kB 9.1 MB/s             
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-082ypm9p/mysqlclient_8c63c04148ca430cacb6612a1c2a0a99/setup.py'"'"'; __file__='"'"'/tmp/pip-install-082ypm9p/mysqlclient_8c63c04148ca430cacb6612a1c2a0a99/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-jzqdbxa6
       cwd: /tmp/pip-install-082ypm9p/mysqlclient_8c63c04148ca430cacb6612a1c2a0a99/
  Complete output (10 lines):
  /bin/sh: 1: mysql_config: not found
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-ins

In [4]:
import mysql.connector

In [25]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="ian",
    database='test'
)

print(mydb)

In [28]:
mycursor = mydb.cursor()

In [24]:
mycursor.execute("CREATE DATABASE test")

In [29]:
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('postthread',)
('postthread2',)
('sys',)
('test',)


In [30]:
mycursor.execute("CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))")

In [36]:
mycursor.execute("CREATE TABLE messages (data TEXT, signer VARCHAR(255), msa_id INT, index INT)")

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'INT)' at line 1